# Hidden Trigger Backdoor Attack Example
This notebook shows how to use the ART implementation of the Hidden Trigger Backdoor attack to poison a model. The goal of the attack is to create poisoned inputs within a target class with no visible trigger that can be used during model finetuning to poison the model. After finetuning, adding a trigger to certain inputs not in the target class will cause the model to reliably misclassify those inputs as the target class. Full details of the attack can be found at https://arxiv.org/abs/1910.00033.

In [1]:
import os, sys
from os.path import abspath

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

import tempfile
# Create a temporary directory for the model checkpoint. Remove the exising one if this cell is rerun
try:
    temp_model_dir.cleanup()
except NameError:
    print("Temporary directory not created yet")
finally:
    temp_model_dir = tempfile.TemporaryDirectory() 
    print("Temporary directory:", temp_model_dir.name)

Temporary directory not created yet
Temporary directory: /tmp/tmpkokqkf5u


## Load Dataset and Model
We will load the CIFAR10 dataset and a pre-trained alexnet model.

In [2]:
from art.utils import load_dataset
import numpy as np
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('cifar10')

In [3]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models, losses

model = models.Sequential()

# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model.add(layers.Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=x_train.shape[1:]))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

loss_object = losses.CategoricalCrossentropy(from_logits=True)

2022-02-07 20:42:09.241104: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 20:42:09.252305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 20:42:09.253212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 20:42:09.254690: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
from art.estimators.classification import TensorFlowV2Classifier

classifier = TensorFlowV2Classifier(
    model=model,
    loss_object=loss_object,
    train_step=train_step,
    nb_classes=10,
    input_shape=(32, 32, 3),
    clip_values=(min_, max_),
)

In [5]:
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)
model.save(temp_model_dir.name + '/htbd_model')

2022-02-07 20:42:14.500283: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-02-07 20:42:14.865093: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


2022-02-07 20:43:31.210986: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpkokqkf5u/htbd_model/assets


In [6]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 76.22%


## Generate the Poison
We now will generate the poison using the hidden trigger backdoor attack. First, we define the target and source classes as well as the backdoor trigger. The target class will be the class we want to insert poisoned data into. The source class will be the class we will add a trigger to in order to cause misclassification into the target.

The backdoor trigger will be a small image patch inserted into the source class images. At test time, we should be able to use this trigger to cause the classifier to misclassify source class images with the trigger added as the target class.

In [7]:
from art.attacks.poisoning.backdoor_attack import PoisoningAttackBackdoor
target = np.array([0,0,0,0,1,0,0,0,0,0])
source = np.array([0,0,0,1,0,0,0,0,0,0])

# Backdoor Trigger Parameters
patch_size = 8
x_shift = 32 - patch_size - 5
y_shift = 32 - patch_size - 5

# Define the backdoor poisoning object. Calling backdoor.poison(x) will insert the trigger into x.
from art.attacks.poisoning import perturbations
def mod(x):
    original_dtype = x.dtype
    x = perturbations.insert_image(x, backdoor_path="../../utils/data/backdoors/htbd.png",
                                   channels_first=False, random=False, x_shift=x_shift, y_shift=y_shift,
                                   size=(patch_size,patch_size), mode='RGB', blend=1)
    return x.astype(original_dtype)
backdoor = PoisoningAttackBackdoor(mod)

Here we run the attack. `eps` controls how much the target images can be perturbed with respect to an l-infinity distance. `feature_layer` dicates with layer's output will be used to define the attack's loss. It can either be the name of the layer or the layer index according to the ART estimator. `poison_percent` controls how many poisoned samples will be generated based on the size of the input data.

The attack will return poisoned inputs of the target class and the indicies in the data that those poisoned inputs should replace.

In [8]:
from art.attacks.poisoning import HiddenTriggerBackdoor
poison_attack = HiddenTriggerBackdoor(classifier, eps=16/255, target=target, source=source, feature_layer=9, backdoor=backdoor, learning_rate=0.01, decay_coeff = .1, decay_iter = 1000, max_iter=3000, batch_size=25, poison_percent=.015)

poison_data, poison_indices = poison_attack.poison(x_train, y_train)
print("Number of poison samples generated:", len(poison_data))

Hidden Trigger:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch:  0 | batch: 0 | i:     0 | LR: 0.01000 | Loss Val: 2852.665 | Loss Avg: 2852.665
Epoch:  0 | batch: 0 | i:   100 | LR: 0.01000 | Loss Val: 272.750 | Loss Avg: 396.859
Epoch:  0 | batch: 0 | i:   200 | LR: 0.01000 | Loss Val: 242.497 | Loss Avg: 325.773
Epoch:  0 | batch: 0 | i:   300 | LR: 0.01000 | Loss Val: 240.480 | Loss Avg: 297.527
Epoch:  0 | batch: 0 | i:   400 | LR: 0.01000 | Loss Val: 231.204 | Loss Avg: 281.951
Epoch:  0 | batch: 0 | i:   500 | LR: 0.01000 | Loss Val: 228.130 | Loss Avg: 272.251
Epoch:  0 | batch: 0 | i:   600 | LR: 0.01000 | Loss Val: 227.712 | Loss Avg: 265.148
Epoch:  0 | batch: 0 | i:   700 | LR: 0.01000 | Loss Val: 224.862 | Loss Avg: 259.692
Epoch:  0 | batch: 0 | i:   800 | LR: 0.01000 | Loss Val: 218.741 | Loss Avg: 255.377
Epoch:  0 | batch: 0 | i:   900 | LR: 0.01000 | Loss Val: 212.148 | Loss Avg: 250.801
Epoch:  0 | batch: 0 | i:  1000 | LR: 0.00100 | Loss Val: 205.692 | Loss Avg: 246.497
Epoch:  0 | batch: 0 | i:  1100 | LR: 0.00100 | Loss

## Finetune the Model
Now, we must finetune the model using the poisoned data and a small number of clean training inputs.  Here, we randomly select an equal number of training inputs from each of the classes.

In [9]:
# Create finetuning dataset
dataset_size = 2500
num_classes = 10
num_per_class = dataset_size/num_classes

poison_dataset_inds = []

for i in range(num_classes):
    class_inds = np.where(np.argmax(y_train,axis=1) == i)[0]
    num_select = int(num_per_class)
    if np.argmax(target) == i:
        num_select = int(num_select - min(num_per_class,len(poison_data)))
        poison_dataset_inds.append(poison_indices)
        
    if num_select != 0:
        poison_dataset_inds.append(np.random.choice(class_inds, num_select, replace=False))
    
poison_dataset_inds = np.concatenate(poison_dataset_inds)

poison_x = np.copy(x_train)
poison_x[poison_indices] = poison_data
poison_x = poison_x[poison_dataset_inds]

poison_y = np.copy(y_train)[poison_dataset_inds]

In [10]:
model = tf.keras.models.load_model(temp_model_dir.name + '/htbd_model')
temp_model_dir.cleanup() # Remove the temporary directory after loading the checkpoint

model.trainable = False
model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [11]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=10,
    input_shape=(32, 32, 3),
    clip_values=(min_, max_),
)
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 76.22%


In [12]:
finetune_model = tf.keras.layers.Dense(10)(model.layers[-2].output)
finetune_model = tf.keras.Model(inputs=model.inputs, outputs=finetune_model)
finetune_model.summary()

lr = 0.5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

finetune_classifier = TensorFlowV2Classifier(
    model=finetune_model,
    loss_object=loss_object,
    train_step=train_step,
    nb_classes=10,
    input_shape=(32, 32, 3),
    clip_values=(min_, max_),
)




Model: "model_18003"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        

In [13]:
trigger_test_inds = np.where(np.all(y_test == source, axis=1))[0]

test_poisoned_samples, test_poisoned_labels  = backdoor.poison(x_test[trigger_test_inds], y_test[trigger_test_inds])


for i in range(4):
    predictions = finetune_classifier.predict(x_test)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on benign test examples: {}%".format(accuracy * 100))
    
    predictions = finetune_classifier.predict(x_test[trigger_test_inds])
    b_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds], axis=1)) / len(trigger_test_inds)
    print("Accuracy on benign trigger test examples: {}%".format(b_accuracy * 100))
    
    predictions = finetune_classifier.predict(test_poisoned_samples)
    p_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(test_poisoned_labels,axis=1)) / len(test_poisoned_labels)
    print("Accuracy on poison trigger test examples: {}%".format(p_accuracy * 100))
    p_success = np.sum(np.argmax(predictions, axis=1) == np.argmax(target)) / len(test_poisoned_labels)
    print("Success on poison trigger test examples: {}%".format(p_success * 100))
    print()
    print("Training Epoch", i)
    if i != 0:
        lr *= 0.1
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    finetune_classifier = TensorFlowV2Classifier(
        model=finetune_model,
        loss_object=loss_object,
        train_step=train_step,
        nb_classes=10,
        input_shape=(32, 32, 3),
        clip_values=(min_, max_),
    )

    finetune_classifier.fit(poison_x, poison_y, nb_epochs=1)


Accuracy on benign test examples: 10.75%
Accuracy on benign trigger test examples: 22.6%
Accuracy on poison trigger test examples: 79.4%
Success on poison trigger test examples: 7.6%

Training Epoch 0
Accuracy on benign test examples: 72.78999999999999%
Accuracy on benign trigger test examples: 37.6%
Accuracy on poison trigger test examples: 5.4%
Success on poison trigger test examples: 54.300000000000004%

Training Epoch 1
Accuracy on benign test examples: 74.59%
Accuracy on benign trigger test examples: 62.5%
Accuracy on poison trigger test examples: 14.299999999999999%
Success on poison trigger test examples: 56.10000000000001%

Training Epoch 2
Accuracy on benign test examples: 74.77000000000001%
Accuracy on benign trigger test examples: 58.9%
Accuracy on poison trigger test examples: 12.8%
Success on poison trigger test examples: 55.400000000000006%

Training Epoch 3


In [14]:
print("Final Performance")
predictions = finetune_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

predictions = finetune_classifier.predict(x_test[trigger_test_inds])
b_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds], axis=1)) / len(trigger_test_inds)
print("Accuracy on benign trigger test examples: {}%".format(b_accuracy * 100))

predictions = finetune_classifier.predict(test_poisoned_samples)
p_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(test_poisoned_labels,axis=1)) / len(test_poisoned_labels)
print("Accuracy on poison trigger test examples: {}%".format(p_accuracy * 100))
p_success = np.sum(np.argmax(predictions, axis=1) == np.argmax(target)) / len(test_poisoned_labels)
print("Success on poison trigger test examples: {}%".format(p_success * 100))

Final Performance
Accuracy on benign test examples: 74.86%
Accuracy on benign trigger test examples: 58.9%
Accuracy on poison trigger test examples: 12.5%
Success on poison trigger test examples: 55.800000000000004%
